# Data Wrangling for Group 1
#### Coded by Tanner Frahm (adapted from Excel formulas written by Scott Wardle) on behalf of Group 1
#### Overall Aim: Wrangle the raw data from the ICD10 mental Chapter to clean it for import into neo4j
#### April 2024 Final Project

In [1]:
# Import necessary modules
import pandas as pd

In [2]:
# Load in the EXCEL file downloaded from ICD 10
df = pd.read_excel(r"C:\Users\tfrahm\Desktop\ICD-10_MIT_2021_Excel_16-March_2021.xlsx",sheet_name = 'SA ICD-10 MIT 2021')

In [3]:
# Filtering relevant data
df = df[df['Chapter_No'] == 'CHAPTER V'] # Cut to just the chapter we want
df = df[df['Valid_ICD10_ClinicalUse'] == 'Y'] # Cut non-relevant rows

In [4]:
# Chunking Strings
df['Chapter_Desc_Short'] = df['Chapter_Desc'].str[:-10]
df['Group_Code_No'] = df['Group_Code'].str[:3]
df['Chapter_Desc_Range'] = df['Chapter_Desc'].str[-9:]

In [5]:
# Create Dictionaries
Group_Desc_Dict = {'Org, including symptomatic, mental disorders': 'Org, incl SX, mtl D/O',
 'Mental and behavioural disorders due to psychoactive substance use': 'mtl & behv D/O due to psyac sub use',
 'Schizophrenia, schizotypal and delusional disorders': 'SZ, SCHT & del D/O',
 'Mood [affective] disorders': 'Mood [affective] D/O',
 'Neurotic, stress-related and somatoform disorders': 'NRT, stress-r & SMTF D/O',
 'Behavioural syndromes associated with physiological disturbances and physical factors': 'behv SYND assoc w/ phys dist & phys fctrs',
 'Disorders of adult personality and behaviour': 'D/O of adult pers & bhvr',
 'Mental retardation': 'mtl retardation',
 'Disorders of psychological development': 'D/O of psych devt',
 'Behavioural and emotional disorders with onset usually occurring in childhood and adolescence': 'behv & emot D/O w/ onst usu occ in chld & ado',
 'Unspecified mental disorder': 'Unspec mtl D/O'}

ICD10_3_Code_Desc_Dict = {"Dementia in Alzheimer's disease (G30.-+)": 'Dem in AD (G30.-+)',
 'Vascular dementia': 'vasc Dem',
 'Dementia in other diseases classified elsewhere': 'Dem in oth dis clf elsw',
 'Unspecified dementia': 'unspec Dem',
 'Org amnesic syndrome, not induced by alcohol and other psychoactive substances': 'Org amn synd, not ind by alc & oth psy subs',
 'Delirium, not induced by alcohol and other psychoactive substances': 'del, not ind by alc & oth psy subs',
 'Other mental disorders due to brain damage and dysfunction and to physical disease': 'oth mntl d/o due to brain dmg & dyfs & to phys dis',
 'Personality and behavioural disorders due to brain disease, damage and dysfunction': 'prsnlty & beh d/o due to brain dis, dmg & dyfs',
 'Unspecified Org or symptomatic mental disorder': 'unspec Org or sympm mntl d/o',
 'Mental and behavioural disorders due to use of alcohol': 'mntl & beh d/o due to use of alc',
 'Mental and behavioural disorders due to use of opioids': 'mntl & beh d/o due to use of opds',
 'Mental and behavioural disorders due to use of cannabinoids': 'mntl & beh d/o due to use of cnbds',
 'Mental and behavioural disorders due to use of sedatives or hypnotics': 'mntl & beh d/o due to use of sdtvs or hyptcs',
 'Mental and behavioural disorders due to use of cocaine': 'mntl & beh d/o due to use of cocn',
 'Mental and behavioural disorders due to use of other stimulants, including caffeine': 'mntl & beh d/o due to use of oth stmlts, inc cfne',
 'Mental and behavioural disorders due to use of hallucinogens': 'mntl & beh d/o due to use of hlcgns',
 'Mental and behavioural disorders due to use of tobacco': 'mntl & beh d/o due to use of tbco',
 'Mental and behavioural disorders due to use of volatile solvents': 'mntl & beh d/o due to use of voltl slvts',
 'Mental and behavioural disorders due to multiple drug use and use of other psychoactive substances': 'mntl & beh d/o due to mltp drug use & use of oth psy subs',
 'Schizophrenia': 'sz',
 'Schizotypal disorder': 'Schizotypal d/o',
 'Persistent delusional disorders': 'Persistent del d/o',
 'Acute and transient psychotic disorders': 'Acute & transient psy d/o',
 'Induced delusional disorder': 'ind del d/o',
 'Schizoaffective disorders': 'szoafv d/o',
 'Other nonOrg psychotic disorders': 'oth nonOrg psy d/o',
 'Unspecified nonOrg psychosis': 'unspec nonOrg psychosis',
 'Manic episode': 'Manic ep',
 'Bipolar affective disorder': 'BP aff d/o',
 'Depressive episode': 'dep ep',
 'Recurrent depressive disorder': 'Recurr dep d/o',
 'Persistent mood [affective] disorders': 'Persistent mood [aff] d/o',
 'Other mood [affective] disorders': 'oth mood [aff] d/o',
 'Unspecified mood [affective] disorder': 'unspec mood [aff] d/o',
 'Phobic anxiety disorders': 'Phobic anx d/o',
 'Other anxiety disorders': 'oth anx d/o',
 'Obsessive-compulsive disorder': 'Obsessive-compulsive d/o',
 'Reaction to severe stress, and adjustment disorders': 'Reaction to sev stress, & adjustment d/o',
 'Dissociative [conversion] disorders': 'Dissociative [conversion] d/o',
 'Somatoform disorders': 'Somatoform d/o',
 'Other neurotic disorders': 'oth neurotic d/o',
 'Eating disorders': 'Eating d/o',
 'NonOrg sleep disorders': 'NonOrg sleep d/o',
 'Sexual dysfunction, not caused by Org disorder or disease': 'Sexual dyfs, not caused by Org d/o or dis',
 'Mental and behavioural disorders associated with the puerperium, not elsewhere classified': 'mntl & beh d/o assctd with the prprm, not elsw clf',
 'Psychological and behavioural factors associated with disorders or diseases classified elsewhere': 'pychlgl & beh factors assctd with d/o or dis clf elsw',
 'Abuse of non-dependence-producing substances': 'Abuse of non-dep-producing subs',
 'Unspecified behavioural syndromes associated with physiological disturbances and physical factors': 'unspec beh synds assctd with physlgcl dstrb & phys factors',
 'Specific personality disorders': 'spec prsnlty d/o',
 'Mixed and other personality disorders': 'Mixed & oth prsnlty d/o',
 'Enduring personality changes, not attributable to brain damage and disease': 'endur prsnlty chgs, not attr to brain dmg & dis',
 'Habit and impulse disorders': 'Habit & impulse d/o',
 'Gender identity disorders': 'Gender identity d/o',
 'Disorders of sexual preference': 'd/o of sexual pref',
 'Psychological and behavioural disorders associated with sexual development and orientation': 'pychlgl & beh d/o assctd with sexual dev & orient',
 'Other disorders of adult personality and behaviour': 'oth d/o of adult prsnlty & beh',
 'Unspecified disorder of adult personality and behaviour': 'unspec d/o of adult prsnlty & beh',
 'Mild mental retardation': 'Mild mntl ret',
 'Moderate mental retardation': 'mod mntl ret',
 'Severe mental retardation': 'sev mntl ret',
 'Profound mental retardation': 'Profound mntl ret',
 'Other mental retardation': 'oth mntl ret',
 'Unspecified mental retardation': 'unspec mntl ret',
 'Specific developmental disorders of speech and language': 'spec dvlmtl d/o of speech & language',
 'Specific developmental disorders of scholastic skills': 'spec dvlmtl d/o of schol skills',
 'Specific developmental disorder of motor function': 'spec dvlmtl d/o of motor function',
 'Mixed specific developmental disorders': 'Mixed spec dvlmtl d/o',
 'Pervasive developmental disorders': 'Pervasive dvlmtl d/o',
 'Other disorders of psychological development': 'oth d/o of pychlgl dev',
 'Unspecified disorder of psychological development': 'unspec d/o of pychlgl dev',
 'Hyperkinetic disorders': 'Hyperkinetic d/o',
 'Conduct disorders': 'Conduct d/o',
 'Mixed disorders of conduct and emotions': 'Mixed d/o of conduct & emotions',
 'Emotional disorders with onset specific to childhood': 'emot d/o with onset spec to chld',
 'Disorders of social functioning with onset specific to childhood and adolescence': 'd/o of social func with onset spec to chld & adol',
 'Tic disorders': 'Tic d/o',
 'Other behavioural and emotional disorders with onset usually occurring in childhood and adolescence': 'oth beh & emot d/o with onset usu occ in chld & adol',
 'Mental disorder, not otherwise specified': 'mntl d/o, not othwise spec'}

WHO_Full_Desc_Dict = {"Dementia in Alzheimer's disease with early onset (G30.0+)": 'Dem in AD with early onset (G30.0+)',
 "Dementia in Alzheimer's disease with late onset (G30.1+)": 'Dem in AD with late onset (G30.1+)',
 "Dementia in Alzheimer's disease, atypical or mixed type (G30.8+)": 'Dem in AD, atyp or mixed type (G30.8+)',
 "Dementia in Alzheimer's disease, unspecified (G30.9+)": 'Dem in AD, unspec (G30.9+)',
 'Vascular dementia of acute onset': 'vasc Dem of acute onset',
 'Multi-infarct dementia': 'Multi-infarct Dem',
 'Subcortical vascular dementia': 'subc vasc Dem',
 'Mixed cortical and subcortical vascular dementia': 'Mixed cortical & subc vasc Dem',
 'Other vascular dementia': 'oth vasc Dem',
 'Vascular dementia, unspecified': 'vasc Dem, unspec',
 "Dementia in Pick's disease (G31.0+)": "Dem in Pick's dis (G31.0+)",
 'Dementia in Creutzfeldt-Jakob disease (A81.0+)': 'Dem in CJ dis (A81.0+)',
 "Dementia in Huntington's disease (G10+)": 'Dem in HNTG dis (G10+)',
 "Dementia in Parkinson's disease (G20+)": 'Dem in PRK dis (G20+)',
 'Dementia in human immunodeficiency virus [HIV] disease (B22.0+)': 'Dem in human imd virus [HIV] dis (B22.0+)',
 'Dementia in other specified diseases classified elsewhere': 'Dem in oth spec dis clf elsw',
 'Unspecified dementia': 'unspec Dem',
 'Org amnesic syndrome, not induced by alcohol and other psychoactive substances': 'Org amn synd, not ind by alc & oth psy subs',
 'Delirium not superimposed on dementia, so described': 'del not sup on Dem, so desc',
 'Delirium superimposed on dementia': 'del sup on Dem',
 'Other delirium': 'oth del',
 'Delirium, unspecified': 'del, unspec',
 'Org hallucinosis': 'Org hallucinosis',
 'Org catatonic disorder': 'Org catatonic d/o',
 'Org delusional [schizophrenia-like] disorder': 'Org del [sz-like] d/o',
 'Org mood [affective] disorders': 'Org mood [aff] d/o',
 'Org anxiety disorder': 'Org anx d/o',
 'Org dissociative disorder': 'Org dissociative d/o',
 'Org emotionally labile [asthenic] disorder': 'Org emot labile [asthenic] d/o',
 'Mild cognitive disorder': 'Mild cognitive d/o',
 'Other specified mental disorders due to brain damage and dysfunction and to physical disease': 'oth spec mntl d/o due to brain dmg & dyfs & to phys dis',
 'Unspecified mental disorder due to brain damage and dysfunction and to physical disease': 'unspec mntl d/o due to brain dmg & dyfs & to phys dis',
 'Org personality disorder': 'Org prsnlty d/o',
 'Postencephalitic syndrome': 'Postencephalitic synd',
 'Postconcussional syndrome': 'Postconcussional synd',
 'Other Org personality and behavioural disorders due to brain disease, damage and dysfunction': 'oth Org prsnlty & beh d/o due to brain dis, dmg & dyfs',
 'Unspecified Org personality and behavioural disorder due to brain disease, damage and dysfunction': 'unspec Org prsnlty & beh d/o due to brain dis, dmg & dyfs',
 'Unspecified Org or symptomatic mental disorder': 'unspec Org or sympm mntl d/o',
 'Mental and behavioural disorders due to use of alcohol, acute intoxication': 'mntl & beh d/o due to use of alc, acute intxcn',
 'Mental and behavioural disorders due to use of alcohol, harmful use': 'mntl & beh d/o due to use of alc, hrmfl use',
 'Mental and behavioural disorders due to use of alcohol, dependence syndrome': 'mntl & beh d/o due to use of alc, dep synd',
 'Mental and behavioural disorders due to use of alcohol, withdrawal state': 'mntl & beh d/o due to use of alc, w/dl state',
 'Mental and behavioural disorders due to use of alcohol, withdrawal state with delirium': 'mntl & beh d/o due to use of alc, w/dl state with del',
 'Mental and behavioural disorders due to use of alcohol, psychotic disorder': 'mntl & beh d/o due to use of alc, psy d/o',
 'Mental and behavioural disorders due to use of alcohol, amnesic syndrome': 'mntl & beh d/o due to use of alc, amn synd',
 'Mental and behavioural disorders due to use of alcohol, residual and late-onset psychotic disorder': 'mntl & beh d/o due to use of alc, res & late-onset psy d/o',
 'Mental and behavioural disorders due to use of alcohol, other mental and behavioural disorders': 'mntl & beh d/o due to use of alc, oth mntl & beh d/o',
 'Mental and behavioural disorders due to use of alcohol, unspecified mental and behavioural disorder': 'mntl & beh d/o due to use of alc, unspec mntl & beh d/o',
 'Mental and behavioural disorders due to use of opioids, acute intoxication': 'mntl & beh d/o due to use of opds, acute intxcn',
 'Mental and behavioural disorders due to use of opioids, harmful use': 'mntl & beh d/o due to use of opds, hrmfl use',
 'Mental and behavioural disorders due to use of opioids, dependence syndrome': 'mntl & beh d/o due to use of opds, dep synd',
 'Mental and behavioural disorders due to use of opioids, withdrawal state': 'mntl & beh d/o due to use of opds, w/dl state',
 'Mental and behavioural disorders due to use of opioids, withdrawal state with delirium': 'mntl & beh d/o due to use of opds, w/dl state with del',
 'Mental and behavioural disorders due to use of opioids, psychotic disorder': 'mntl & beh d/o due to use of opds, psy d/o',
 'Mental and behavioural disorders due to use of opioids, amnesic syndrome': 'mntl & beh d/o due to use of opds, amn synd',
 'Mental and behavioural disorders due to use of opioids, residual and late-onset psychotic disorder': 'mntl & beh d/o due to use of opds, res & late-onset psy d/o',
 'Mental and behavioural disorders due to use of opioids, other mental and behavioural disorders': 'mntl & beh d/o due to use of opds, oth mntl & beh d/o',
 'Mental and behavioural disorders due to use of opioids, unspecified mental and behavioural disorder': 'mntl & beh d/o due to use of opds, unspec mntl & beh d/o',
 'Mental and behavioural disorders due to use of cannabinoids, acute intoxication': 'mntl & beh d/o due to use of cnbds, acute intxcn',
 'Mental and behavioural disorders due to use of cannabinoids, harmful use': 'mntl & beh d/o due to use of cnbds, hrmfl use',
 'Mental and behavioural disorders due to use of cannabinoids, dependence syndrome': 'mntl & beh d/o due to use of cnbds, dep synd',
 'Mental and behavioural disorders due to use of cannabinoids, withdrawal state': 'mntl & beh d/o due to use of cnbds, w/dl state',
 'Mental and behavioural disorders due to use of cannabinoids, withdrawal state with delirium': 'mntl & beh d/o due to use of cnbds, w/dl state with del',
 'Mental and behavioural disorders due to use of cannabinoids, psychotic disorder': 'mntl & beh d/o due to use of cnbds, psy d/o',
 'Mental and behavioural disorders due to use of cannabionoids, amnesic syndrome': 'mntl & beh d/o due to use of cnbds, amn synd',
 'Mental and behavioural disorders due to use of cannabinoids, residual and late-onset psychotic disorder': 'mntl & beh d/o due to use of cnbds, res & late-onset psy d/o',
 'Mental and behavioural disorders due to use of cannabinoids, other mental and behavioural disorders': 'mntl & beh d/o due to use of cnbds, oth mntl & beh d/o',
 'Mental and behavioural disorders due to use of cannabinoids, unspecified mental and behavioural disorder': 'mntl & beh d/o due to use of cnbds, unspec mntl & beh d/o',
 'Mental and behavioural disorders due to use of sedatives or hypnotics, acute intoxication': 'mntl & beh d/o due to use of sdtvs or hyptcs, acute intxcn',
 'Mental and behavioural disorders due to use of sedatives or hypnotics, harmful use': 'mntl & beh d/o due to use of sdtvs or hyptcs, hrmfl use',
 'Mental and behavioural disorders due to use of sedatives or hypnotics, dependence syndrome': 'mntl & beh d/o due to use of sdtvs or hyptcs, dep synd',
 'Mental and behavioural disorders due to use of sedatives or hypnotics, withdrawal state': 'mntl & beh d/o due to use of sdtvs or hyptcs, w/dl state',
 'Mental and behavioural disorders due to use of sedatives or hypnotics, withdrawal state with delirium': 'mntl & beh d/o due to use of sdtvs or hyptcs, w/dl state with del',
 'Mental and behavioural disorders due to use of sedatives or hypnotics, psychotic disorder': 'mntl & beh d/o due to use of sdtvs or hyptcs, psy d/o',
 'Mental and behavioural disorders due to use of sedatives or hypnotics, amnesic syndrome': 'mntl & beh d/o due to use of sdtvs or hyptcs, amn synd',
 'Mental and behavioural disorders due to use of sedatives or hypnotics, residual and late-onset psychotic disorder': 'mntl & beh d/o due to use of sdtvs or hyptcs, res & late-onset psy d/o',
 'Mental and behavioural disorders due to use of sedatives or hypnotics, other mental and behavioural disorders': 'mntl & beh d/o due to use of sdtvs or hyptcs, oth mntl & beh d/o',
 'Mental and behavioural disorders due to use of sedatives or hypnotics, unspecified mental and behavioural disorder': 'mntl & beh d/o due to use of sdtvs or hyptcs, unspec mntl & beh d/o',
 'Mental and behavioural disorders due to use of cocaine, acute intoxication': 'mntl & beh d/o due to use of cocn, acute intxcn',
 'Mental and behavioural disorders due to use of cocaine, harmful use': 'mntl & beh d/o due to use of cocn, hrmfl use',
 'Mental and behavioural disorders due to use of cocaine, dependence syndrome': 'mntl & beh d/o due to use of cocn, dep synd',
 'Mental and behavioural disorders due to use of cocaine, withdrawal state': 'mntl & beh d/o due to use of cocn, w/dl state',
 'Mental and behavioural disorders due to use of cocaine, withdrawal state with delirium': 'mntl & beh d/o due to use of cocn, w/dl state with del',
 'Mental and behavioural disorders due to use of cocaine, psychotic disorder': 'mntl & beh d/o due to use of cocn, psy d/o',
 'Mental and behavioural disorders due to use of cocaine, amnesic syndrome': 'mntl & beh d/o due to use of cocn, amn synd',
 'Mental and behavioural disorders due to use of cocaine, residual and late-onset psychotic disorder': 'mntl & beh d/o due to use of cocn, res & late-onset psy d/o',
 'Mental and behavioural disorders due to use of cocaine, other mental and behavioural disorders': 'mntl & beh d/o due to use of cocn, oth mntl & beh d/o',
 'Mental and behavioural disorders due to use of cocaine, unspecified mental and behavioural disorder': 'mntl & beh d/o due to use of cocn, unspec mntl & beh d/o',
 'Mental and behavioural disorders due to use of other stimulants, including caffeine, acute intoxication': 'mntl & beh d/o due to use of oth stmlts, inc cfne, acute intxcn',
 'Mental and behavioural disorders due to use of other stimulants, including caffeine, harmful use': 'mntl & beh d/o due to use of oth stmlts, inc cfne, hrmfl use',
 'Mental and behavioural disorders due to use of other stimulants, including caffeine, dependence syndrome': 'mntl & beh d/o due to use of oth stmlts, inc cfne, dep synd',
 'Mental and behavioural disorders due to use of other stimulants, including caffeine, withdrawal state': 'mntl & beh d/o due to use of oth stmlts, inc cfne, w/dl state',
 'Mental and behavioural disorders due to use of other stimulants, including caffeine, withdrawal state with delirium': 'mntl & beh d/o due to use of oth stmlts, inc cfne, w/dl state with del',
 'Mental and behavioural disorders due to use of other stimulants, including caffeine, psychotic disorder': 'mntl & beh d/o due to use of oth stmlts, inc cfne, psy d/o',
 'Mental and behavioural disorders due to use of other stimulants, including caffeine, amnesic syndrome': 'mntl & beh d/o due to use of oth stmlts, inc cfne, amn synd',
 'Mental and behavioural disorders due to use of other stimulants, including caffeine, residual and late-onset psychotic disorder': 'mntl & beh d/o due to use of oth stmlts, inc cfne, res & late-onset psy d/o',
 'Mental and behavioural disorders due to use of other stimulants, including caffeine, other mental and behavioural disorders': 'mntl & beh d/o due to use of oth stmlts, inc cfne, oth mntl & beh d/o',
 'Mental and behavioural disorders due to use of other stimulants, including caffeine, unspecified mental and behavioural disorder': 'mntl & beh d/o due to use of oth stmlts, inc cfne, unspec mntl & beh d/o',
 'Mental and behavioural disorders due to use of hallucinogens, acute intoxication': 'mntl & beh d/o due to use of hlcgns, acute intxcn',
 'Mental and behavioural disorders due to use of hallucinogens, harmful use': 'mntl & beh d/o due to use of hlcgns, hrmfl use',
 'Mental and behavioural disorders due to use of hallucinogens, dependence syndrome': 'mntl & beh d/o due to use of hlcgns, dep synd',
 'Mental and behavioural disorders due to use of hallucinogens, withdrawal state': 'mntl & beh d/o due to use of hlcgns, w/dl state',
 'Mental and behavioural disorders due to use of hallucinogens, withdrawal state with delirium': 'mntl & beh d/o due to use of hlcgns, w/dl state with del',
 'Mental and behavioural disorders due to use of hallucinogens, psychotic disorder': 'mntl & beh d/o due to use of hlcgns, psy d/o',
 'Mental and behavioural disorders due to use of hallucinogens, amnesic syndrome': 'mntl & beh d/o due to use of hlcgns, amn synd',
 'Mental and behavioural disorders due to use of hallucinogens, residual and late-onset psychotic disorder': 'mntl & beh d/o due to use of hlcgns, res & late-onset psy d/o',
 'Mental and behavioural disorders due to use of hallucinogens, other mental and behavioural disorders': 'mntl & beh d/o due to use of hlcgns, oth mntl & beh d/o',
 'Mental and behavioural disorders due to use of hallucinogens, unspecified mental and behavioural disorder': 'mntl & beh d/o due to use of hlcgns, unspec mntl & beh d/o',
 'Mental and behavioural disorders due to use of tobacco, acute intoxication': 'mntl & beh d/o due to use of tbco, acute intxcn',
 'Mental and behavioural disorders due to use of tobacco, harmful use': 'mntl & beh d/o due to use of tbco, hrmfl use',
 'Mental and behavioural disorders due to use of tobacco, dependence syndrome': 'mntl & beh d/o due to use of tbco, dep synd',
 'Mental and behavioural disorders due to use of tobacco, withdrawal state': 'mntl & beh d/o due to use of tbco, w/dl state',
 'Mental and behavioural disorders due to use of tobacco, withdrawal state with delirium': 'mntl & beh d/o due to use of tbco, w/dl state with del',
 'Mental and behavioural disorders due to use of tobacco, psychotic disorder': 'mntl & beh d/o due to use of tbco, psy d/o',
 'Mental and behavioural disorders due to use of tobacco, amnesic syndrome': 'mntl & beh d/o due to use of tbco, amn synd',
 'Mental and behavioural disorders due to use of tobacco, residual and late-onset psychotic disorder': 'mntl & beh d/o due to use of tbco, res & late-onset psy d/o',
 'Mental and behavioural disorders due to use of tobacco, other mental and behavioural disorders': 'mntl & beh d/o due to use of tbco, oth mntl & beh d/o',
 'Mental and behavioural disorders due to use of tobacco, unspecified mental and behavioural disorder': 'mntl & beh d/o due to use of tbco, unspec mntl & beh d/o',
 'Mental and behavioural disorders due to use of volatile solvents, acute intoxication': 'mntl & beh d/o due to use of voltl slvts, acute intxcn',
 'Mental and behavioural disorders due to use of volatile solvents, harmful use': 'mntl & beh d/o due to use of voltl slvts, hrmfl use',
 'Mental and behavioural disorders due to use of volatile solvents, dependence syndrome': 'mntl & beh d/o due to use of voltl slvts, dep synd',
 'Mental and behavioural disorders due to use of volatile solvents, withdrawal state': 'mntl & beh d/o due to use of voltl slvts, w/dl state',
 'Mental and behavioural disorders due to use of volatile solvents, withdrawal state with delirium': 'mntl & beh d/o due to use of voltl slvts, w/dl state with del',
 'Mental and behavioural disorders due to use of volatile solvents, psychotic disorder': 'mntl & beh d/o due to use of voltl slvts, psy d/o',
 'Mental and behavioural disorders due to use of volatile solvents, amnesic syndrome': 'mntl & beh d/o due to use of voltl slvts, amn synd',
 'Mental and behavioural disorders due to use of volatile solvents, residual and late-onset psychotic disorder': 'mntl & beh d/o due to use of voltl slvts, res & late-onset psy d/o',
 'Mental and behavioural disorders due to use of volatile solvents, other mental and behavioural disorders': 'mntl & beh d/o due to use of voltl slvts, oth mntl & beh d/o',
 'Mental and behavioural disorders due to use of volatile solvents, unspecified mental and behavioural disorder': 'mntl & beh d/o due to use of voltl slvts, unspec mntl & beh d/o',
 'Mental and behavioural disorders due to multiple drug use and use of other psychoactive substances, acute intoxication': 'mntl & beh d/o due to mltp drug use & use of oth psy subs, acute intxcn',
 'Mental and behavioural disorders due to multiple drug use and use of other psychoactive substances, harmful use': 'mntl & beh d/o due to mltp drug use & use of oth psy subs, hrmfl use',
 'Mental and behavioural disorders due to multiple drug use and use of other psychoactive substances, dependence syndrome': 'mntl & beh d/o due to mltp drug use & use of oth psy subs, dep synd',
 'Mental and behavioural disorders due to multiple drug use and use of other psychoactive substances, withdrawal state': 'mntl & beh d/o due to mltp drug use & use of oth psy subs, w/dl state',
 'Mental and behavioural disorders due to multiple drug use and use of other psychoactive substances, withdrawal state with delirium': 'mntl & beh d/o due to mltp drug use & use of oth psy subs, w/dl state with del',
 'Mental and behavioural disorders due to multiple drug use and use of other psychoactive substances, other psychotic disorder': 'mntl & beh d/o due to mltp drug use & use of oth psy subs, oth psy d/o',
 'Mental and behavioural disorders due to multiple drug use and use of other psychoactive substances, amnesic syndrome': 'mntl & beh d/o due to mltp drug use & use of oth psy subs, amn synd',
 'Mental and behavioural disorders due to multiple drug use and use of other psychoactive substances, residual and late-onset other psychotic disorder': 'mntl & beh d/o due to mltp drug use & use of oth psy subs, res & late-onset oth psy d/o',
 'Mental and behavioural disorders due to multiple drug use and use of other psychoactive substances, other mental and behavioural disorders': 'mntl & beh d/o due to mltp drug use & use of oth psy subs, oth mntl & beh d/o',
 'Mental and behavioural disorders due to multiple drug use and use of other psychoactive substances, unspecified mental and behavioural disorder': 'mntl & beh d/o due to mltp drug use & use of oth psy subs, unspec mntl & beh d/o',
 'Paranoid schizophrenia': 'Paranoid sz',
 'Hebephrenic schizophrenia': 'Hebephrenic sz',
 'Catatonic schizophrenia': 'Catatonic sz',
 'Undifferentiated schizophrenia': 'Undifferentiated sz',
 'Post-schizophrenic depression': 'Post-sz dep',
 'Residual schizophrenia': 'res sz',
 'Simple schizophrenia': 'Simple sz',
 'Other schizophrenia': 'oth sz',
 'Schizophrenia, unspecified': 'sz, unspec',
 'Schizotypal disorder': 'Schizotypal d/o',
 'Delusional disorder': 'del d/o',
 'Other persistent delusional disorders': 'oth persistent del d/o',
 'Persistent delusional disorder, unspecified': 'Persistent del d/o, unspec',
 'Acute polymorphic psychotic disorder without symptoms of schizophrenia': 'Acute plymrpc psy d/o w/o symp of sz',
 'Acute polymorphic psychotic disorder with symptoms of schizophrenia': 'Acute plymrpc psy d/o with symp of sz',
 'Acute schizophrenia-like psychotic disorder': 'Acute sz-like psy d/o',
 'Other acute predominantly delusional psychotic disorders': 'oth acute predom del psy d/o',
 'Other acute and transient psychotic disorders': 'oth acute & transient psy d/o',
 'Acute and transient psychotic disorder, unspecified': 'Acute & transient psy d/o, unspec',
 'Induced delusional disorder': 'ind del d/o',
 'Schizoaffective disorder, manic type': 'szoafv d/o, manic type',
 'Schizoaffective disorder, depressive type': 'szoafv d/o, dep type',
 'Schizoaffective disorder, mixed type': 'szoafv d/o, mixed type',
 'Other schizoaffective disorders': 'oth szoafv d/o',
 'Schizoaffective disorder, unspecified': 'szoafv d/o, unspec',
 'Other nonOrg psychotic disorders': 'oth nonOrg psy d/o',
 'Unspecified nonOrg psychosis': 'unspec nonOrg psychosis',
 'Hypomania': 'hypm',
 'Mania without psychotic symptoms': 'Mania w/o psy symp',
 'Mania with psychotic symptoms': 'Mania with psy symp',
 'Other manic episodes': 'oth manic eps',
 'Manic episode, unspecified': 'Manic ep, unspec',
 'Bipolar affective disorder, current episode hypomanic': 'BP aff d/o, curr ep hypmc',
 'Bipolar affective disorder, current episode manic without psychotic symptoms': 'BP aff d/o, curr ep manic w/o psy symp',
 'Bipolar affective disorder, current episode manic with psychotic symptoms': 'BP aff d/o, curr ep manic with psy symp',
 'Bipolar affective disorder, current episode mild or moderate depression': 'BP aff d/o, curr ep mild or mod dep',
 'Bipolar affective disorder, current episode severe depression without psychotic symptoms': 'BP aff d/o, curr ep sev dep w/o psy symp',
 'Bipolar affective disorder, current episode severe depression with psychotic symptoms': 'BP aff d/o, curr ep sev dep with psy symp',
 'Bipolar affective disorder, current episode mixed': 'BP aff d/o, curr ep mixed',
 'Bipolar affective disorder, currently in remission': 'BP aff d/o, currly in rem',
 'Other bipolar affective disorders': 'oth BP aff d/o',
 'Bipolar affective disorder, unspecified': 'BP aff d/o, unspec',
 'Mild depressive episode': 'Mild dep ep',
 'Moderate depressive episode': 'mod dep ep',
 'Severe depressive episode without psychotic symptoms': 'sev dep ep w/o psy symp',
 'Severe depressive episode with psychotic symptoms': 'sev dep ep with psy symp',
 'Other depressive episodes': 'oth dep eps',
 'Depressive episode, unspecified': 'dep ep, unspec',
 'Recurrent depressive disorder, current episode mild': 'Recurr dep d/o, curr ep mild',
 'Recurrent depressive disorder, current episode moderate': 'Recurr dep d/o, curr ep mod',
 'Recurrent depressive disorder, current episode severe without psychotic symptoms': 'Recurr dep d/o, curr ep sev w/o psy symp',
 'Recurrent depressive disorder, current episode severe with psychotic symptoms': 'Recurr dep d/o, curr ep sev with psy symp',
 'Recurrent depressive disorder, currently in remission': 'Recurr dep d/o, currly in rem',
 'Other recurrent depressive disorders': 'oth recurr dep d/o',
 'Recurrent depressive disorder, unspecified': 'Recurr dep d/o, unspec',
 'Cyclothymia': 'Cyclothymia',
 'Dysthymia': 'Dysthymia',
 'Other persistent mood [affective] disorders': 'oth persistent mood [aff] d/o',
 'Persistent mood [affective] disorder, unspecified': 'Persistent mood [aff] d/o, unspec',
 'Other single mood [affective] disorders': 'oth single mood [aff] d/o',
 'Other recurrent mood [affective] disorders': 'oth recurr mood [aff] d/o',
 'Other specified mood [affective] disorders': 'oth spec mood [aff] d/o',
 'Unspecified mood [affective] disorder': 'unspec mood [aff] d/o',
 'Agoraphobia': 'Agoraphobia',
 'Social phobias': 'Social phobias',
 'Specific (isolated) phobias': 'spec (isolated) phobias',
 'Other phobic anxiety disorders': 'oth phobic anx d/o',
 'Phobic anxiety disorder, unspecified': 'Phobic anx d/o, unspec',
 'Panic disorder [episodic paroxysmal anxiety]': 'Panic d/o [episodic paroxysmal anx]',
 'Generalized anxiety disorder': 'Generalized anx d/o',
 'Mixed anxiety and depressive disorder': 'Mixed anx & dep d/o',
 'Other mixed anxiety disorders': 'oth mixed anx d/o',
 'Other specified anxiety disorders': 'oth spec anx d/o',
 'Anxiety disorder, unspecified': 'anx d/o, unspec',
 'Predominantly obsessional thoughts or ruminations': 'predom obsessional thoughts or ruminations',
 'Predominantly compulsive acts [obsessional rituals]': 'predom compulsive acts [obsessional rituals]',
 'Mixed obsessional thoughts and acts': 'Mixed obsessional thoughts & acts',
 'Other obsessive-compulsive disorders': 'oth obsessive-compulsive d/o',
 'Obsessive-compulsive disorder, unspecified': 'Obsessive-compulsive d/o, unspec',
 'Acute stress reaction': 'Acute stress reaction',
 'Post-traumatic stress disorder': 'Post-traumatic stress d/o',
 'Adjustment disorders': 'Adjustment d/o',
 'Other reactions to severe stress': 'oth reactions to sev stress',
 'Reaction to severe stress, unspecified': 'Reaction to sev stress, unspec',
 'Dissociative amnesia': 'Dissociative amnesia',
 'Dissociative fugue': 'Dissociative fugue',
 'Dissociative stupor': 'Dissociative stupor',
 'Trance and possession disorders': 'Trance & possession d/o',
 'Dissociative motor disorders': 'Dissociative motor d/o',
 'Dissociative convulsions': 'Dissociative convulsions',
 'Dissociative anaesthesia and sensory loss': 'Dissociative anaesthesia & sensory loss',
 'Mixed dissociative [conversion] disorders': 'Mixed dissociative [conversion] d/o',
 'Other dissociative [conversion] disorders': 'oth dissociative [conversion] d/o',
 'Dissociative [conversion] disorder, unspecified': 'Dissociative [conversion] d/o, unspec',
 'Somatization disorder': 'Somatization d/o',
 'Undifferentiated somatoform disorder': 'Undifferentiated somatoform d/o',
 'Hypochondriacal disorder': 'Hypochondriacal d/o',
 'Somatoform autonomic dysfunction': 'Somatoform autonomic dyfs',
 'Persistent somatoform pain disorder': 'Persistent somatoform pain d/o',
 'Other somatoform disorders': 'oth somatoform d/o',
 'Somatoform disorder, unspecified': 'Somatoform d/o, unspec',
 'Neurasthenia': 'Neurasthenia',
 'Depersonalization-derealization syndrome': 'Depersonalization-derealization synd',
 'Other specified neurotic disorders': 'oth spec neurotic d/o',
 'Neurotic disorder, unspecified': 'Neurotic d/o, unspec',
 'Anorexia nervosa': 'Anorexia nervosa',
 'Atypical anorexia nervosa': 'atyp anorexia nervosa',
 'Bulimia nervosa': 'Bulimia nervosa',
 'Atypical bulimia nervosa': 'atyp bulimia nervosa',
 'Overeating associated with other psychological disturbances': 'Overeating assctd with oth pychlgl dstrb',
 'Vomiting associated with other psychological disturbances': 'Vomiting assctd with oth pychlgl dstrb',
 'Other eating disorders': 'oth eating d/o',
 'Eating disorder, unspecified': 'Eating d/o, unspec',
 'NonOrg insomnia': 'NonOrg insomnia',
 'NonOrg hypersomnia': 'NonOrg hypersomnia',
 'NonOrg disorder of the sleep-wake schedule': 'NonOrg d/o of the sleep-wake schedule',
 'Sleepwalking [somnambulism]': 'Sleepwalking [somnambulism]',
 'Sleep terrors [night terrors]': 'Sleep terrors [night terrors]',
 'Nightmares': 'Nightmares',
 'Other nonOrg sleep disorders': 'oth nonOrg sleep d/o',
 'NonOrg sleep disorder, unspecified': 'NonOrg sleep d/o, unspec',
 'Lack or loss of sexual desire': 'Lack or loss of sexual desire',
 'Sexual aversion and lack of sexual enjoyment': 'Sexual aversion & lack of sexual enjoyment',
 'Failure of genital response': 'Failure of genital response',
 'Orgasmic dysfunction': 'Orgasmic dyfs',
 'Premature ejaculation': 'Premature ejaculation',
 'NonOrg vaginismus': 'NonOrg vaginismus',
 'NonOrg dyspareunia': 'NonOrg dyspareunia',
 'Excessive sexual drive': 'Excessive sexual drive',
 'Other sexual dysfunction, not caused by Org disorder or disease': 'oth sexual dyfs, not caused by Org d/o or dis',
 'Unspecified sexual dysfunction, not caused by Org disorder or disease': 'unspec sexual dyfs, not caused by Org d/o or dis',
 'Mild mental and behavioural disorders associated with the puerperium, not elsewhere classified': 'Mild mntl & beh d/o assctd with the prprm, not elsw clf',
 'Severe mental and behavioural disorders associated with the puerperium, not elsewhere classified': 'sev mntl & beh d/o assctd with the prprm, not elsw clf',
 'Other mental and behavioural disorders associated with the puerperium, not elsewhere classified': 'oth mntl & beh d/o assctd with the prprm, not elsw clf',
 'Puerperal mental disorder, unspecified': 'Puerperal mntl d/o, unspec',
 'Psychological and behavioural factors associated with disorders or diseases classified elsewhere': 'pychlgl & beh factors assctd with d/o or dis clf elsw',
 'Abuse of non-dependence-producing substances': 'Abuse of non-dep-producing subs',
 'Unspecified behavioural syndromes associated with physiological disturbances and physical factors': 'unspec beh synds assctd with physlgcl dstrb & phys factors',
 'Paranoid personality disorder': 'Paranoid prsnlty d/o',
 'Schizoid personality disorder': 'Schizoid prsnlty d/o',
 'Dissocial personality disorder': 'Dissocial prsnlty d/o',
 'Emotionally unstable personality disorder': 'emot unstable prsnlty d/o',
 'Histrionic personality disorder': 'Histrionic prsnlty d/o',
 'Anankastic personality disorder': 'Anankastic prsnlty d/o',
 'Anxious [avoidant] personality disorder': 'Anxious [avoidant] prsnlty d/o',
 'Dependent personality disorder': 'Dependent prsnlty d/o',
 'Other specific personality disorders': 'oth spec prsnlty d/o',
 'Personality disorder, unspecified': 'prsnlty d/o, unspec',
 'Mixed and other personality disorders': 'Mixed & oth prsnlty d/o',
 'Enduring personality change after catastrophic experience': 'endur prsnlty change after cat exp',
 'Enduring personality change after psychiatric illness': 'endur prsnlty change after psych illness',
 'Other enduring personality changes': 'oth endur prsnlty chgs',
 'Enduring personality change, unspecified': 'endur prsnlty change, unspec',
 'Pathological gambling': 'path gambling',
 'Pathological fire-setting [pyromania]': 'path fire-setting [pyromania]',
 'Pathological stealing [kleptomania]': 'path stealing [kleptomania]',
 'Trichotillomania': 'Trichotillomania',
 'Other habit and impulse disorders': 'oth habit & impulse d/o',
 'Habit and impulse disorder, unspecified': 'Habit & impulse d/o, unspec',
 'Transsexualism': 'Transsexualism',
 'Dual-role transvestism': 'Dual-role transvestism',
 'Gender identity disorder of childhood': 'Gender identity d/o of chld',
 'Other gender identity disorders': 'oth gender identity d/o',
 'Gender identity disorder, unspecified': 'Gender identity d/o, unspec',
 'Fetishism': 'Fetishism',
 'Fetishistic transvestism': 'Fetishistic transvestism',
 'Exhibitionism': 'Exhibitionism',
 'Voyeurism': 'Voyeurism',
 'Paedophilia': 'Paedophilia',
 'Sadomasochism': 'Sadomasochism',
 'Multiple disorders of sexual preference': 'mltp d/o of sexual pref',
 'Other disorders of sexual preference': 'oth d/o of sexual pref',
 'Disorder of sexual preference, unspecified': 'd/o of sexual pref, unspec',
 'Sexual maturation disorder': 'Sexual maturation d/o',
 'Egodystonic sexual orientation': 'Egodystonic sexual orient',
 'Sexual relationship disorder': 'Sexual relationship d/o',
 'Other psychosexual development disorders': 'oth psychosexual dev d/o',
 'Psychosexual development disorder, unspecified': 'Psychosexual dev d/o, unspec',
 'Elaboration of physical symptoms for psychological reasons': 'Elaboration of phys symp for pychlgl reasons',
 'Intentional production or feigning of symptoms or disabilities, either physical or psychological [factitious disorder]': 'int prod or feigning of symp or dis, either phys or pychlgl [factitious d/o]',
 'Other specified disorders of adult personality and behaviour': 'oth spec d/o of adult prsnlty & beh',
 'Unspecified disorder of adult personality and behaviour': 'unspec d/o of adult prsnlty & beh',
 'Mild mental retardation with the statement of no, or minimal, impairment of behaviour': 'Mild mntl ret with the stmt of no, or min, imp of beh',
 'Mild mental retardation, significant impairment of behaviour requiring attention or treatment': 'Mild mntl ret, sig imp of beh req att or tx',
 'Mild mental retardation, other impairments of behaviour': 'Mild mntl ret, oth imps of beh',
 'Mild mental retardation without mention of impairment of behaviour': 'Mild mntl ret w/o mention of imp of beh',
 'Moderate mental retardation with the statement of no, or minimal, impairment of behaviour': 'mod mntl ret with the stmt of no, or min, imp of beh',
 'Moderate mental retardation, significant impairment of behaviour requiring attention or treatment': 'mod mntl ret, sig imp of beh req att or tx',
 'Moderate mental retardation, other impairments of behaviour': 'mod mntl ret, oth imps of beh',
 'Moderate mental retardation without mention of impairment of behaviour': 'mod mntl ret w/o mention of imp of beh',
 'Severe mental retardation with the statement of no, or minimal, impairment of behaviour': 'sev mntl ret with the stmt of no, or min, imp of beh',
 'Severe mental retardation, significant impairment of behaviour requiring attention or treatment': 'sev mntl ret, sig imp of beh req att or tx',
 'Severe mental retardation, other impairments of behaviour': 'sev mntl ret, oth imps of beh',
 'Severe mental retardation without mention of impairment of behaviour': 'sev mntl ret w/o mention of imp of beh',
 'Profound mental retardation with the statement of no, or minimal, impairment of behaviour': 'Profound mntl ret with the stmt of no, or min, imp of beh',
 'Profound mental retardation, significant impairment of behaviour requiring attention or treatment': 'Profound mntl ret, sig imp of beh req att or tx',
 'Profound mental retardation, other impairments of behaviour': 'Profound mntl ret, oth imps of beh',
 'Profound mental retardation without mention of impairment of behaviour': 'Profound mntl ret w/o mention of imp of beh',
 'Other mental retardation with the statement of no, or minimal, impairment of behaviour': 'oth mntl ret with the stmt of no, or min, imp of beh',
 'Other mental retardation, significant impairment of behaviour requiring attention or treatment': 'oth mntl ret, sig imp of beh req att or tx',
 'Other mental retardation, other impairments of behaviour': 'oth mntl ret, oth imps of beh',
 'Other mental retardation without mention of impairment of behaviour': 'oth mntl ret w/o mention of imp of beh',
 'Unspecified mental retardation with the statement of no, or minimal, impairment of behaviour': 'unspec mntl ret with the stmt of no, or min, imp of beh',
 'Unspecified mental retardation, significant impairment of behaviour requiring attention or treatment': 'unspec mntl ret, sig imp of beh req att or tx',
 'Unspecified mental retardation, other impairments of behaviour': 'unspec mntl ret, oth imps of beh',
 'Unspecified mental retardation without mention of impairment of behaviour': 'unspec mntl ret w/o mention of imp of beh',
 'Specific speech articulation disorder': 'spec speech articulation d/o',
 'Expressive language disorder': 'Expressive language d/o',
 'Receptive language disorder': 'Receptive language d/o',
 'Acquired aphasia with epilepsy [Landau-Kleffner]': 'Acquired aphasia with epilepsy [Landau-Kleffner]',
 'Other developmental disorders of speech and language': 'oth dvlmtl d/o of speech & language',
 'Developmental disorder of speech and language, unspecified': 'dvlmtl d/o of speech & language, unspec',
 'Specific reading disorder': 'spec reading d/o',
 'Specific spelling disorder': 'spec spelling d/o',
 'Specific disorder of arithmetical skills': 'spec d/o of arithmetical skills',
 'Mixed disorder of scholastic skills': 'Mixed d/o of schol skills',
 'Other developmental disorders of scholastic skills': 'oth dvlmtl d/o of schol skills',
 'Developmental disorder of scholastic skills, unspecified': 'dvlmtl d/o of schol skills, unspec',
 'Specific developmental disorder of motor function': 'spec dvlmtl d/o of motor function',
 'Mixed specific developmental disorders': 'Mixed spec dvlmtl d/o',
 'Childhood autism': 'chld autism',
 'Atypical autism': 'atyp autism',
 "Rett's syndrome": "Rett's synd",
 'Other childhood disintegrative disorder': 'oth chld disintegrative d/o',
 'Overactive disorder associated with mental retardation and stereotyped movements': 'ovract d/o assctd with mntl ret & stereotyped mvmts',
 "Asperger's syndrome": "Asperger's synd",
 'Other pervasive developmental disorders': 'oth pervasive dvlmtl d/o',
 'Pervasive developmental disorder, unspecified': 'Pervasive dvlmtl d/o, unspec',
 'Other disorders of psychological development': 'oth d/o of pychlgl dev',
 'Unspecified disorder of psychological development': 'unspec d/o of pychlgl dev',
 'Disturbance of activity and attention': 'Disturbance of activity & att',
 'Hyperkinetic conduct disorder': 'Hyperkinetic conduct d/o',
 'Other hyperkinetic disorders': 'oth hyperkinetic d/o',
 'Hyperkinetic disorder, unspecified': 'Hyperkinetic d/o, unspec',
 'Conduct disorder confined to the family context': 'Conduct d/o confined to the family context',
 'Unsocialized conduct disorder': 'Unsocialized conduct d/o',
 'Socialized conduct disorder': 'Socialized conduct d/o',
 'Oppositional defiant disorder': 'Oppositional defiant d/o',
 'Other conduct disorders': 'oth conduct d/o',
 'Conduct disorder, unspecified': 'Conduct d/o, unspec',
 'Depressive conduct disorder': 'dep conduct d/o',
 'Other mixed disorders of conduct and emotions': 'oth mixed d/o of conduct & emotions',
 'Mixed disorder of conduct and emotions, unspecified': 'Mixed d/o of conduct & emotions, unspec',
 'Separation anxiety disorder of childhood': 'Separation anx d/o of chld',
 'Phobic anxiety disorder of childhood': 'Phobic anx d/o of chld',
 'Social anxiety disorder of childhood': 'Social anx d/o of chld',
 'Sibling rivalry disorder': 'Sibling rivalry d/o',
 'Other childhood emotional disorders': 'oth chld emot d/o',
 'Childhood emotional disorder, unspecified': 'chld emot d/o, unspec',
 'Elective mutism': 'Elective mutism',
 'Reactive attachment disorder of childhood': 'Reactive att d/o of chld',
 'Disinhibited attachment disorder of childhood': 'Disinhibited att d/o of chld',
 'Other childhood disorders of social functioning': 'oth chld d/o of social func',
 'Childhood disorder of social functioning, unspecified': 'chld d/o of social func, unspec',
 'Transient tic disorder': 'Transient tic d/o',
 'Chronic motor or vocal tic disorder': 'Chronic motor or vocal tic d/o',
 'Combined vocal and multiple motor tic disorder [de La Tourette]': 'Combined vocal & mltp motor tic d/o [de La Tourette]',
 'Other tic disorders': 'oth tic d/o',
 'Tic disorder, unspecified': 'Tic d/o, unspec',
 'NonOrg enuresis': 'NonOrg enuresis',
 'NonOrg encopresis': 'NonOrg encopresis',
 'Feeding disorder of infancy and childhood': 'Feeding d/o of infancy & chld',
 'Pica of infancy and childhood': 'Pica of infancy & chld',
 'Stereotyped movement disorders': 'Stereotyped movement d/o',
 'Stuttering [stammering]': 'Stuttering [stammering]',
 'Cluttering': 'Cluttering',
 'Other specified behavioural and emotional disorders with onset usually occurring in childhood and adolescence': 'oth spec beh & emot d/o with onset usu occ in chld & adol',
 'Unspecified behavioural and emotional disorders with onset usually occurring in childhood and adolescence': 'unspec beh & emot d/o with onset usu occ in chld & adol',
 'Mental disorder, not otherwise specified': 'mntl d/o, not othwise spec'}

In [6]:
# Create New Columns
df['Group_Desc_Short'] = df['Group_Desc'].replace(Group_Desc_Dict)
df['ICD10_3_Code_Desc_Short'] = df['ICD10_3_Code_Desc'].replace(ICD10_3_Code_Desc_Dict)
df['WHO_Full_Desc_Short'] = df['WHO_Full_Desc'].replace(WHO_Full_Desc_Dict)

In [7]:
# Export to CSV
df.to_csv(r"C:\Users\tfrahm\Desktop\ICD-10_MIT_2021_Wrangled.csv")